In [7]:
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
import pymongo
import spacy
import re
from collections import Counter


In [8]:
def noticias_mercurio_dia(año, mes, dia):
    start = dt(año, mes, dia, 0, 0, 0)
    
    end = dt(año, mes, dia + 1, 0, 0, 0)
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client["noticias"]
    collection = db["mercurio_raw"]
    noticias_mercurio = [noticia for noticia in collection.find( {'fechaPublicacion': {'$lt': end, '$gte': start}})]
    return noticias_mercurio



In [9]:
def noticias_mostrador_dia(año, mes, dia):
    start = dt(año, mes, dia, 0, 0, 0)
    end = dt(año, mes, dia + 1, 0, 0, 0)
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client["noticias"]
    collection = db["mostrador_raw"]
    noticias_mostrador = [noticia for noticia in collection.find( {'fecha publicacion': {'$lt': end, '$gte': start}})]
    return noticias_mostrador

In [10]:
def taglist_mostrador(noticias):
    tags = [noticia["tags"] for noticia in noticias]
    return [tag for sublist in tags for tag in sublist]

In [11]:
nlp = spacy.load("es_core_news_lg")


In [12]:
def doc_mercurio(noticias):
    text = " ".join([(noticia["titulo"] + " " + noticia["bajada"] + " " + noticia["texto"]).replace(u'\xa0', u' ') for noticia in noticias])
    doc = nlp(text)
    return doc

In [13]:
def common_proper_nouns(document):
    importantes = ["PROPN"]
    lemas = [ w.lemma_ for w in document  if w.pos_ in importantes]
    cuontalemas = Counter(lemas)
    cuenta = Counter.most_common(cuontalemas)
    return {k:v for (k,v) in cuenta}

In [14]:
1

1

In [15]:
1


1

In [16]:

dia_mercurio = noticias_mercurio_dia(2020, 6, 16)
documentoc = doc_mercurio(dia_mercurio)
common_proper_nouns(documentoc)


{'&': 39,
 'Chile': 37,
 'Gobierno': 36,
 'Salud': 36,
 'covid-19': 28,
 'Paris': 26,
 'Presidente': 22,
 'Estados': 20,
 'Unidos': 18,
 'Estado': 17,
 'Ministerio': 14,
 'Barcelona': 13,
 'Región': 13,
 'Minsal': 13,
 'Enrique': 13,
 'José': 12,
 'Emergencia': 12,
 'Santiago': 12,
 'Carabineros': 12,
 'IFE': 12,
 'Pastora': 12,
 'Emmy': 12,
 'Luis': 11,
 'Juan': 11,
 'Twitter': 11,
 'Piñera': 11,
 'Metropolitana': 10,
 'Cámara': 10,
 'Berkhoff': 10,
 'Social': 10,
 'Sebastián': 10,
 'Vidal': 9,
 'Universidad': 9,
 'Interior': 9,
 'Emol': 9,
 'Familiar': 9,
 'Carlos': 9,
 'Nicaragua': 9,
 'Trump': 9,
 'Leganés': 8,
 'Nacional': 8,
 'Ejecutivo': 8,
 'San': 8,
 'Biobío': 8,
 'Blumel': 8,
 'América': 8,
 'Sur': 8,
 'Desarrollo': 8,
 'Ingreso': 8,
 'Alberto': 8,
 'Carmona': 8,
 'Shaya': 8,
 'Sifup': 8,
 'García': 8,
 'Nueva': 8,
 'York': 8,
 'Tiltil': 8,
 'Messi': 7,
 'OMS': 7,
 'Perú': 7,
 'EE.UU.': 7,
 'Senado': 7,
 'PS': 7,
 'Romario': 7,
 'Costa': 7,
 'Rica': 7,
 'República': 7,
 'Paul

In [17]:
fecha = (2020, 7, 15)
mercurio_ = noticias_mercurio_dia(2020, 7, 15)
document_ = doc_mercurio(mercurio_)
sustantivos_comunes_ = common_proper_nouns(document_)
sustantivos_comunes_

{'Gobierno': 107,
 'Chile': 56,
 'Cámara': 51,
 'Senado': 37,
 'China': 37,
 '&': 36,
 'AFP': 33,
 'RN': 33,
 'Presidente': 32,
 'Diputados': 28,
 'Salud': 27,
 'Hong': 25,
 'Kong': 25,
 'Estado': 23,
 'Congreso': 23,
 'Heard': 22,
 'covid-19': 21,
 'Valparaíso': 21,
 'Interior': 20,
 'Huawei': 19,
 'UDI': 19,
 'Moneda': 18,
 'Perú': 18,
 'Ministerio': 18,
 'Blumel': 18,
 'Jorge': 17,
 'Unidos': 16,
 'EE.UU.': 16,
 'Partido': 16,
 'Carabineros': 16,
 'Nacional': 16,
 'Estados': 15,
 'Piñera': 15,
 'Central': 14,
 'Sebastián': 14,
 'Madrid': 14,
 'Paris': 14,
 'Apple': 14,
 'Sala': 13,
 'Durán': 13,
 'Gonzalo': 13,
 'Trump': 12,
 'Santiago': 12,
 'The': 12,
 'Juan': 12,
 'Alexis': 11,
 'Twitter': 11,
 'Washington': 11,
 'Miguel': 11,
 'Briones': 11,
 'City': 11,
 'Andrés': 11,
 'Celis': 11,
 'Rubilar': 10,
 'Mandatario': 10,
 'Hacienda': 10,
 'James': 10,
 'Depp': 10,
 'Barcelona': 10,
 'Pablo': 10,
 'José': 9,
 'España': 9,
 'Brasil': 9,
 'Diego': 9,
 'Murphy': 9,
 'Ignacio': 9,
 'Nico

In [19]:
fecha = (2020, 7, 10)
mercurio_ = noticias_mercurio_dia(*fecha)
document_ = doc_mercurio(mercurio_)
sustantivos_comunes_ = common_proper_nouns(document_)
sustantivos_comunes_

{'Chile': 119,
 'Universidad': 117,
 'Gobierno': 88,
 'Salud': 58,
 'Brasil': 51,
 'AFP': 41,
 'Ministerio': 40,
 '&': 38,
 'Presidente': 35,
 'Cámara': 30,
 'covid-19': 30,
 'Colo': 29,
 'Federal': 29,
 'RN': 26,
 'Heard': 24,
 'UDI': 23,
 'Nacional': 23,
 'Depp': 23,
 'Paris': 21,
 'Estado': 20,
 'Santa': 20,
 'México': 19,
 'Estados': 19,
 'Unidos': 18,
 'Carabineros': 17,
 'Ejecutivo': 17,
 'Educación': 16,
 'Católica': 16,
 'Italia': 16,
 'Park': 16,
 'Sebastián': 15,
 'Diputados': 15,
 'Colegio': 15,
 'Mandatario': 15,
 'Sofía': 15,
 'OMS': 15,
 'Evópoli': 14,
 'Colombia': 14,
 'Blumel': 14,
 'The': 13,
 'Santiago': 13,
 'Contraloría': 13,
 'Antofagasta': 13,
 'Argentina': 13,
 'Banco': 13,
 'Minsal': 12,
 'Interior': 12,
 'República': 12,
 'Pontificia': 12,
 'Servicio': 12,
 'Inter': 12,
 'Congreso': 11,
 'Médico': 11,
 'Enrique': 11,
 'Constitución': 11,
 'Hipódromo': 11,
 'Gonzalo': 11,
 'China': 11,
 'Región': 10,
 'José': 10,
 'Atención': 10,
 'Fernández': 10,
 'Sala': 10,
 